In [2]:
!pip install geopy pandas faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 10.1 MB/s eta 0:00:00


In [3]:
import pkg_resources
installed_packages = {d.project_name for d in pkg_resources.working_set}
print(installed_packages)

{'attrs', 'conda-package-handling', 'ipython', 'jupyter-server', 'zstandard', 'websocket-client', 'Brotli', 'debugpy', 'conda-package-streaming', 'backports.tarfile', 'tinycss2', 'frozendict', 'tomli', 'truststore', 'conda-content-trust', 'pycosat', 'cffi', 'idna', 'jsonpointer', 'pycparser', 'wheel', 'pyogrio', 'parso', 'matplotlib-inline', 'rfc3986-validator', 'pluggy', 'stack-data', 'wcwidth', 'h11', 'Send2Trash', 'argon2-cffi', 'soupsieve', 'jupyter-server-terminals', 'MarkupSafe', 'python-dateutil', 'bleach', 'jaraco.text', 'ruamel.yaml', 'overrides', 'prompt-toolkit', 'pandas', 'jaraco.functools', 'tzdata', 'inflect', 'faker', 'geographiclib', 'jupyterlab-pygments', 'mistune', 'notebook-shim', 'PySocks', 'jupyterlab-server', 'menuinst', 'psutil', 'pyproj', 'jupyterlab', 'six', 'platformdirs', 'importlib-metadata', 'asttokens', 'pexpect', 'certifi', 'json5', 'jaraco.collections', 'ipykernel', 'urllib3', 'PyYAML', 'jaraco.context', 'conda', 'jupyter-core', 'python-json-logger', 'ru

/tmp/ipykernel_26930/2113228596.py:1: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


In [7]:
# notebooks/generate_places_data.ipynb

# 1. Required Imports
import pandas as pd
import random
from faker import Faker
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
import time
import re
import os

# 2. Configuration
fake = Faker('pt_BR')
geolocator = Nominatim(user_agent="iotclima_data_generator_v2")
OUTPUT_DIR = 'data'
OUTPUT_FILE = os.path.join(OUTPUT_DIR, 'places_data.csv')

# 3. City Data
def get_cities_data():
    return {
        "Vale do Jaguaribe": {
            "state": "Ceará", "country": "Brazil", "cities": [
                "Alto Santo", "Ereré", "Iracema", "Jaguaretama", "Jaguaribara",
                "Jaguaribe", "Limoeiro do Norte", "Morada Nova", "Palhano",
                "Pereiro", "Potiretama", "Quixeré", "Russas", "São João do Jaguaribe",
                "Tabuleiro do Norte"
            ]},
        "Sertão dos Inhamuns": {
            "state": "Ceará", "country": "Brazil", "cities": [
                "Aiuaba", "Arneiroz", "Parambu", "Quiterianópolis", "Tauá"
            ]},
        "Sertão de Canindé": {
            "state": "Ceará", "country": "Brazil", "cities": [
                "Boa Viagem", "Canindé", "Caridade", "Itatira", "Madalena", "Paramoti"
            ]},
        "Maciço do Baturité": {
            "state": "Ceará", "country": "Brazil", "cities": [
                "Acarape", "Aracoiaba", "Aratuba", "Barreira", "Baturité",
                "Capistrano", "Guaramiranga", "Itapiúna", "Mulungu", "Ocara",
                "Pacoti", "Palmácia", "Redenção"
            ]},
        "Litoral Oeste/Vale do Curu": {
            "state": "Ceará", "country": "Brazil", "cities": [
                "Apuiarés", "General Sampaio", "Irauçuba", "Itapajé", "Miraíma",
                "Pentecoste", "Tejuçuoca", "Tururu", "Umirim", "Uruburetama"
            ]},
        "Litoral Norte": {
            "state": "Ceará", "country": "Brazil", "cities": [
                "Acaraú", "Barroquinha", "Bela Cruz", "Camocim", "Chaval",
                "Cruz", "Granja", "Jijoca de Jericoacoara", "Marco", "Morrinhos",
                "Uruoca", "Martinópole"
            ]},
        "Serra da Ibiapaba": {
            "state": "Ceará", "country": "Brazil", "cities": [
                "Carnaubal", "Croatá", "Guaraciaba do Norte", "Ibiapina", "Ipu",
                "São Benedito", "Tianguá", "Ubajara", "Viçosa do Ceará"
            ]},
        "Grande Fortaleza": {
            "state": "Ceará", "country": "Brazil", "cities": [
                "Chorozinho", "São Luís do Curu"
            ]}
    }

# 4. Data Generation Functions
def generate_city_abbreviation(city_name: str) -> str:
    cleaned_name = re.sub(r'\b(do|da|dos|das|de|di|e)\b', '', city_name, flags=re.IGNORECASE)
    cleaned_name = re.sub(r'[^a-zA-Z]', '', cleaned_name).upper()
    
    if len(cleaned_name) >= 3:
        return cleaned_name[:3]
    elif len(cleaned_name) > 0:
        return cleaned_name
    return "XXX"

def get_geocoded_data(city_name, state, country, geolocator):
    query = f"{city_name}, {state}, {country}"
    try:
        location = geolocator.geocode(query, timeout=10)
        if location:
            raw_address_dict = location.raw.get('address', {})
            formatted_full_address = location.raw.get('display_name', query)

            canonical_city_name = raw_address_dict.get('city') or raw_address_dict.get('town') or city_name

            country_code = raw_address_dict.get('country_code', 'BR').upper()
            state_abbr = raw_address_dict.get('state_abbr', state[:2].upper())
            derived_place_code = f"{country_code}-{state_abbr}-{generate_city_abbreviation(canonical_city_name)}"
            
            return location.latitude, location.longitude, raw_address_dict, canonical_city_name, derived_place_code, formatted_full_address
        
        print(f"Warning: No geocoding result for '{query}'.")
        return None, None, None, None, None, None
    
    except GeocoderTimedOut:
        print(f"Error: Geocoding service timed out for '{query}'. Retrying in 1 second.")
        time.sleep(1)
        return get_geocoded_data(city_name, state, country, geolocator)
    except GeocoderServiceError as e:
        print(f"Error: Geocoding service error for '{query}': {e}. Skipping city.")
        return None, None, None, None, None, None
    except Exception as e:
        print(f"An unexpected error occurred during geocoding for '{query}': {e}. Skipping city.")
        return None, None, None, None, None, None

# 5. Data Generation and Saving
places_records = []
places_per_city = 1

all_cities_data = get_cities_data()

os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"Starting data generation for {len(all_cities_data)} regions...")

for region, data in all_cities_data.items():
    state = data['state']
    country = data['country']
    for city_name_full in data['cities']:
        latitude, longitude, raw_address_dict, canonical_city_name, derived_place_code, formatted_full_address = get_geocoded_data(city_name_full, state, country, geolocator)
        time.sleep(1.1)

        if latitude is None or longitude is None:
            print(f"Skipping data generation for '{city_name_full}' due to missing geocoding data.")
            continue

        address_to_use = formatted_full_address
        
        if not address_to_use:
            if isinstance(raw_address_dict, dict):
                parts_from_dict = [
                    raw_address_dict.get('road'),
                    raw_address_dict.get('house_number'),
                    raw_address_dict.get('suburb'),
                    raw_address_dict.get('city') or raw_address_dict.get('town'),
                    raw_address_dict.get('state'),
                    raw_address_dict.get('postcode'),
                    raw_address_dict.get('country')
                ]
                address_to_use = ", ".join(filter(None, parts_from_dict))
            
            if not address_to_use:
                address_to_use = f"{city_name_full}, {state}, {country}"


        for i in range(places_per_city):
            place_record = {
                'name': f"{derived_place_code}",
                'description': f"Meteorological station located in {canonical_city_name}, {state}, {country}. Georeferenced location.",
                'address': address_to_use,
                'city': canonical_city_name,
                'state': state,
                'country': country,
                'status': 1,
                'type': 3,
                'latitude': latitude,
                'longitude': longitude,
            }
            places_records.append(place_record)
            print(f"Generated data for '{place_record['name']}' ({derived_place_code}) in {canonical_city_name}")

df = pd.DataFrame(places_records)
df.to_csv(OUTPUT_FILE, index=False)

print(f"\nData generation completed. Total records: {len(df)}. Saved to {OUTPUT_FILE}")

Starting data generation for 8 regions...
Generated data for 'BR-CE-ALT_1' (BR-CE-ALT) in Alto Santo
Generated data for 'BR-CE-ERE_1' (BR-CE-ERE) in Ereré
Generated data for 'BR-CE-IRA_1' (BR-CE-IRA) in Iracema
Generated data for 'BR-CE-JAG_1' (BR-CE-JAG) in Jaguaretama
Generated data for 'BR-CE-JAG_1' (BR-CE-JAG) in Jaguaribara
Generated data for 'BR-CE-JAG_1' (BR-CE-JAG) in Jaguaribe
Generated data for 'BR-CE-LIM_1' (BR-CE-LIM) in Limoeiro do Norte
Generated data for 'BR-CE-MOR_1' (BR-CE-MOR) in Morada Nova
Generated data for 'BR-CE-PAL_1' (BR-CE-PAL) in Palhano
Generated data for 'BR-CE-PER_1' (BR-CE-PER) in Pereiro
Generated data for 'BR-CE-POT_1' (BR-CE-POT) in Potiretama
Generated data for 'BR-CE-QUI_1' (BR-CE-QUI) in Quixeré
Generated data for 'BR-CE-RUS_1' (BR-CE-RUS) in Russas
Generated data for 'BR-CE-SOJ_1' (BR-CE-SOJ) in São João do Jaguaribe
Generated data for 'BR-CE-TAB_1' (BR-CE-TAB) in Tabuleiro do Norte
Generated data for 'BR-CE-AIU_1' (BR-CE-AIU) in Aiuaba
Generated d